In [1]:
import os
import requests
from openai import OpenAI
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pip show tiktoken

Name: tiktoken
Version: 0.9.0
Summary: tiktoken is a fast BPE tokeniser for use with OpenAI's models
Home-page: https://github.com/openai/tiktoken
Author: Shantanu Jain
Author-email: shantanu@openai.com
License: MIT License

Copyright (c) 2022 OpenAI, Shantanu Jain

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULA

In [17]:
!pip uninstall elasticsearch --yes

Found existing installation: elasticsearch 9.0.2
Uninstalling elasticsearch-9.0.2:
  Successfully uninstalled elasticsearch-9.0.2


In [12]:
!pip show elasticsearch

Name: elasticsearch
Version: 9.0.2
Summary: Python client for Elasticsearch
Home-page: https://github.com/elastic/elasticsearch-py
Author: 
Author-email: Elastic Client Library Maintainers <client-libs@elastic.co>
License-Expression: Apache-2.0
Location: /home/codespace/miniconda3/lib/python3.13/site-packages
Requires: elastic-transport, python-dateutil, typing-extensions
Required-by: 


In [19]:
!pip install elasticsearch==9.0.2

  Using cached elasticsearch-9.0.2-py3-none-any.whl.metadata (8.4 kB)
Using cached elasticsearch-9.0.2-py3-none-any.whl (914 kB)


In [14]:
client = OpenAI(base_url="https://api.proxyapi.ru/openai/v1",
                api_key=os.environ['KEY'])

In [ ]:
resp = client.chat.completions.create(
    model='gpt-4o-mini',
    max_completion_tokens=5,
    messages=[{'role': 'user', 'content': 'как приготовить лосося?'}]
)
resp.choices[0].messege.content

In [5]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
es_client = Elasticsearch('http://localhost:9200'
                         )

In [8]:
es_client.info()

ObjectApiResponse({'name': '175042a80b3b', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'rfGryj34Siur4yiauXF7Lg', 'version': {'number': '9.0.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '0a58bc1dc7a4ae5412db66624aab968370bd44ce', 'build_date': '2025-05-28T10:06:37.834829258Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

In [9]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = 'course-questions'

In [13]:
es_client.info()

ObjectApiResponse({'name': '175042a80b3b', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'rfGryj34Siur4yiauXF7Lg', 'version': {'number': '9.0.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '0a58bc1dc7a4ae5412db66624aab968370bd44ce', 'build_date': '2025-05-28T10:06:37.834829258Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

In [ ]:
es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [14]:
query = 'How do execute a command on a Kubernetes pod?'

In [15]:
search_query = {
    "size": 1,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text",],
                    "type": "best_fields"
                }
            },
        }
    }
}

In [16]:
response = es_client.search(index=index_name, body=search_query)

In [17]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'WhA8b5cBrtm5cMNyYeP7',
  '_score': 44.50556,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}}]

In [18]:
query = 'How do copy a file to a Docker container?'

In [19]:
def search_elasic(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    results = [hit['_source'] for hit in response['hits']['hits']]
    
    return results

In [20]:
results = search_elasic(query)
results

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [21]:
def get_promt(query, results):

    context = ''
    for doc in results:
        context += context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"
    
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()
    
    promt = prompt_template.format(question=query, context=context)

    return promt

In [22]:
get_promt(query, results)

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\n    Use only the facts from the CONTEXT when answering the QUESTION.\n\n    QUESTION: How do copy a file to a Docker container?\n\n    CONTEXT:\n    Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I 

In [23]:
len(get_promt(query, results))

2981

In [24]:
def use_llm(query, promt):
    client = OpenAI(base_url="https://api.proxyapi.ru/openai/v1",
                    api_key=os.environ['KEY'])
    resp = client.chat.completions.create(
        model='gpt-4o-mini',
        max_completion_tokens=5,
        messages=[{'role': 'user', 'content': promt}]
    )
    return resp.choices[0].messege.content

In [27]:
def ask_llm(query):
    results = search_elasic(query)
    promt = get_promt(query, results)
    answer = use_llm(query, promt)
    return answer

In [34]:
ask_llm('How do copy a file to a Docker container?')

KeyError: 'KEY'

In [32]:
!pip install tiktoken

In [33]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

ModuleNotFoundError: No module named 'tiktoken'